In [0]:
COLAB = True # NOTE change me if not running on Colab :D

## Importing libs

In [0]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [0]:
%matplotlib inline
sns.set() # apply the seaborn defaults to plotted figures (e.g. theme, scaling, color palette), instead of matplotlib's

In [8]:
if COLAB:
    try:
        # Colab only
        %tensorflow_version 2.x
    except Exception:
        pass

TensorFlow is already loaded. Please restart the runtime to change versions.


In [9]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

1.15.0


In [0]:
import warnings
from time import time, strftime

In [11]:
strftime('%X')

'20:18:24'